In [1]:
### Google Colab configuration ###
# from google.colab import drive
# drive.mount("/content/drive/")
# %cd "/content/drive/My Drive/Colab Notebooks/Facial sentiment analysis"

Mounted at /content/drive/
/content/drive/My Drive/Colab Notebooks/Facial sentiment analysis


In [3]:
import cvlib as cv    # !pip install cvlib    # Colab doesn't have cvlib
import io, os, json
import numpy as np
import cv2, requests
from PIL import Image
import matplotlib.pyplot as plt

In [7]:
%%time
json_list = ("sad_img_url.json", 
             "shocked_img_url.json", 
             "happy_img_url.json")

folder_list = ("sad_folder", 
               "shocked_folder", 
               "happy_folder")

for json_file, folder in zip(json_list, folder_list):
    with open(json_file, "r") as f:
        loaded_json = json.load(f)
        print(f"{folder} is filling...")
        os.mkdir(folder)
        for i in range(len(loaded_json)):
            try:
                img_link = loaded_json[i].popitem()[1]
                img = Image.open(io.BytesIO(requests.get(img_link).content))
                img = np.array(img)
                # OpenCV requires 3D images to detect faces:
                img = cv2.cvtColor(img, code=8) if len(img.shape) != 3 else img
                
                # detect, crop and save the faces
                faces, confidence = cv.detect_face(img, threshold=0.80)
                for face_num, face in enumerate(faces, start=1):
                    (startX, startY) = (face[0], face[1])    # top left corner
                    (endX, endY) = (face[2], face[3])    # bottom right corner
                    cropped_frame = np.copy(img[startY:endY, startX:endX])
                    # keep only large-enough (24-pixel) frames
                    if cropped_frame.shape[0] > 23 or cropped_frame.shape[1] > 23:
                        cropped_frame = cv2.cvtColor(cropped_frame, code=6)
                        to_path = f"{folder}/p{i}_f{face_num}.jpg"    #sample file: "p15_f2.jpg" (picture15, face2)
                        Image.fromarray(cropped_frame).save(to_path) 
            except Exception as e:
                print(e, f"(error at index: {i})")
                continue
        print(f"{folder} is filled!\n")
del json_list, folder_list, loaded_json

shocked_folder is filling...
OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
 (error at index: 573)
OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
 (error at index: 1840)
Invalid URL '': No schema supplied. Perhaps you meant http://? (error at index: 1845)
OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
 (error at index: 2045)
OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
 (error at index: 2086)
cannot identify image file <_i